# Введение в рекомендательные системы

## Коллаборативная фильтрация

В этом задании мы закончим имплементацию коллаборативной фильтрации.

Для этого - выполним действия, необходимые для создания матрицы рейтингов

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [184]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

In [185]:
filepath = './data/user_ratedmovies.dat'
df_rates = pd.read_csv(filepath, sep='\t')

In [186]:
filepath = './data/movies.dat'
df_movies = pd.read_csv(filepath, sep='\t', encoding='iso-8859-1')

# Перекодируем ID фильмов и пользователей

In [187]:
from sklearn.preprocessing import LabelEncoder

In [188]:
enc_user = LabelEncoder()
enc_mov = LabelEncoder()

In [189]:
enc_user = enc_user.fit(df_rates.userID.values)
enc_mov = enc_mov.fit(df_rates.movieID.values)

In [190]:
idx = df_movies.loc[:, 'id'].isin(df_rates.movieID)
df_movies = df_movies.loc[idx]

In [191]:
df_rates.loc[:, 'userID'] = enc_user.transform(df_rates.loc[:, 'userID'].values)

print(df_rates.loc[:10, 'userID'] )

df_rates.loc[:, 'movieID'] = enc_mov.transform(df_rates.loc[:, 'movieID'].values)
df_movies.loc[:, 'id'] = enc_mov.transform(df_movies.loc[:, 'id'].values)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
Name: userID, dtype: int64


## Матрица рейтингов

In [10]:
from scipy.sparse import coo_matrix, csr_matrix

In [11]:
R = coo_matrix((df_rates.rating.values, (df_rates.userID.values, df_rates.movieID.values)))

In [12]:
R

<2113x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 855598 stored elements in COOrdinate format>

## Похожесть между пользователями

В дальнейшем нам будет удобнее работать с форматом `Compressed Sparse Row matrix`. К счастью переформатировать полученную нами матрицу можно одной командой:

In [13]:
R = R.tocsr()

Теперь, например, рейтинги для первого пользователя можно достать так:

In [14]:
user_1 = R[0]
user_1

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 55 stored elements in Compressed Sparse Row format>

Так как вы возможно не работали с разреженным форматом матриц, устроим небольшой ликбез.

Первым делом, надо понадобится вектор для другого пользователя:

In [15]:
user_2 = R[1]
user_2

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 468 stored elements in Compressed Sparse Row format>

Мы можем сравнивать элементы с 0

In [16]:
user_1_rated = (user_1 != 0)
user_1_rated

<1x10109 sparse matrix of type '<class 'numpy.bool_'>'
	with 55 stored elements in Compressed Sparse Row format>

Можем их "индексировать"

In [17]:
user_1[user_1_rated]

matrix([[1. , 4.5, 4. , 2. , 4. , 4.5, 3.5, 5. , 3.5, 2. , 4. , 3. , 4.5,
         0.5, 4.5, 4. , 3.5, 4.5, 4. , 2.5, 4. , 4. , 4. , 4.5, 2.5, 2. ,
         1.5, 4. , 4. , 4.5, 3. , 3. , 4.5, 3.5, 4.5, 1.5, 3. , 3. , 3.5,
         3.5, 3. , 2.5, 3.5, 4. , 0.5, 4. , 3.5, 4.5, 3.5, 4.5, 5. , 3.5,
         3.5, 3.5, 4.5]])

Можем считать количество ненулевых элементов

In [18]:
user_1.nnz

55

Можем умножать 2 разреженных вектора поэлементно:

In [19]:
(user_1).multiply(user_2)

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 14 stored elements in Compressed Sparse Row format>

и скалярно

In [20]:
user_1.dot(user_2.T)[0, 0]

216.75

И превращать разреженную матрицу (вектор) в плотную

In [21]:
user_1_dense = user_1.toarray()
user_1_dense

array([[0., 0., 1., ..., 0., 0., 0.]])

Этого ликбеза вам будет должно быть достаточно, чтобы реализовать функцию расчета похожести между парой пользователей $u$ и $v$:

$$ s_{uv} = \frac{\sum\limits_{i \in I_u\cap I_v} R_{ui} R_{vi}}{\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{ui}^2}}\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{vi}^2}}}$$

Давайте будем считать, что если количество фильмов которые пользователь $u$ и $v$ посмотрели вместе $<= 2$, то их косинусная мера равна 0.0


### _Решение_

In [30]:
import math

def cosine_similarity_pair_users(u, v):
    intersect = (u != 0).multiply(v != 0)
    if intersect.nnz <= 2:
        return 0.0
    
    return u[intersect].dot(v[intersect].T)[0, 0] / math.sqrt(u[intersect].dot(u[intersect].T)[0, 0]) / math.sqrt(v[intersect].dot(v[intersect].T)[0, 0])

In [31]:
answer1 = round(cosine_similarity_pair_users(R[146], R[239]),3)
print(answer1)

0.923


Введите значение answer1 на странице https://www.coursera.org/learn/python-for-data-science/exam/fSPxW/sozdaniie-riekomiendatiel-noi-sistiemy

## Функция нахождения пользователей, схожих с данным. 

Реализуйте функцию <font color = "blue">similar_users</font>(u, R, n_neigbours) которая принимает на входе
* Индекс пользователя
* Матрицу рейтингов
* Количество ближайших соседей 
и возвращает отсортированный массив пользователей (сортировка по неубыванию), максимально похожих на данного. Для сортировки используйте np.argsort без параметров. (https://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html) (Сам пользователь будет в этом списке на первом месте). Эту функцию вы сможете использовать далее. 

In [143]:
def similar_users(u, R, n_neigbours):
    vals = []
    for i, n in enumerate(R):
        vals.append((i, cosine_similarity_pair_users(R[u], R[i])))
    
    arr = np.array([v for _, v in vals])
    
    return np.argsort(arr)[::-1][:n_neigbours]

def similar_users2(u, R, n_neigbours):
    vals = []
    for i, n in enumerate(R):
        vals.append((i, cosine_similarity_pair_users(u, R[i])))
    
    arr = np.array([v for _, v in vals])
    
    return np.argsort(arr)[::-1][:n_neigbours]

In [45]:
answer2 = np.array2string(similar_users(42, R, 10)).replace(' ','').replace('[','').replace(']','')

len(answer2)

29

In [46]:
answer2

'42281633724815262065016921506'

Введите значение answer2 без кавычек  на странице https://www.coursera.org/learn/python-for-data-science/exam/fSPxW/sozdaniie-riekomiendatiel-noi-sistiemy. Это будет строка из 29 символов, которая начинается на 42.

## Функция прогнозирования рейтинга

Реализуйте функцию <font color = "blue">rate_items_user</font>(u, R, n_neigbours), которая принимает на входе:
* Индекс пользователя
* Матрицу рейтингов
* Количество ближайших соседей <font color = "red">(Теперь обратите внимание, несмотря на то, что каждый пользователь - ближайший сосед самому себе, в расчетах он использоваться не должен)</font>

и возвращает вектор с предсказанными рейтингами по всем фильмам для этого пользователя

Для того, чтобы считать прогноз по рейтингу мы воспользуемся упрощенной формулой из лекции:

$$ \hat{R}_{ui} = \frac{\sum_{v \in N(u)} s_{uv}R_{vi}}{\sum_{v \in N(u)} \left| s_{uv}\right|} $$


### _Решение_

In [168]:
def rate_items_user(u, R, n_neigbours):
    predictions = np.empty(R.shape[1])
    
    neib = similar_users(u, R, n_neigbours+1)[1:]
    
    ss = np.array([cosine_similarity_pair_users(R[u], R[v]) for v in neib])
    sm = np.sum(np.abs(ss))
    
    for i in range(R.shape[1]):
        predictions[i] = ss.dot(R[neib,i].toarray())
    
    return predictions/sm


def rate_items_user2(u, R, n_neigbours):
    predictions = np.empty(R.shape[1])
    
    dists = np.array([cosine_similarity_pair_users(R[u], v) for v in R])
    
    
    
    for i in range(R.shape[1]):
        inds = np.arange(R.shape[0])[(R[:, i] != 0).toarray().flatten()]
        
        down = np.argsort(dists[inds])[:min(n_neigbours, inds.size)]
        
        #print(R2.shape)
        neib = inds[down]
    
        ss = dists[neib]
        sm = np.sum(np.abs(ss))

        predictions[i] = ss.dot(R[neib,i].toarray())
    
    return predictions/sm



In [161]:
(R[:, 3] != 0).toarray().flatten().shape

(2113,)

В качестве ответа к этому заданию верните 5 идентификаторов фильмов с наивысшим предсказанным рейтингом для пользователя с id 19 (20-я строчка в матрице рейтингов).

* Для усреднения используйте 30 ближайших соседей
* Среди этих 5-и фильмов не должно быть ранее просмотренных фильмов

Т.е. предсказанные рейтинги можно получить так:
R_hat = <font color = "blue">rate_items_user</font>(20, R, n_neigbours=30). При сортировке фильмов по рейтингу используйте функцию <font color = "blue">argsort </font> без параметров.

### _Решение_

In [195]:
id_ = 20

R_hat = rate_items_user(id_, R, n_neigbours=30)

In [196]:
print((R[id_,:]==0).toarray())

unwatched = np.arange(R.shape[1])[(R[id_,:]==0).toarray()[0]]
#print(unwatched)
print(unwatched.shape)

print(R_hat[:15])

print(np.sort(R_hat)[::-1])

top = np.array([v for v in np.argsort(R_hat)[::-1] if v in unwatched])

print(top.shape)

top5 = top[:5]

[[ True  True  True ...  True  True  True]]
(10076,)
[1.04983968 0.56640284 0.09990098 0.         0.         0.
 0.19999455 0.         0.         0.         0.14987514 0.
 0.         0.         0.        ]
[2.5834227  2.46640169 2.30002146 ... 0.         0.         0.        ]
(10076,)


## Генерация ответа

In [197]:
answer3 = ', '.join(str(i) for i in top5)

answer3

'2614, 306, 343, 5573, 6720'

Полученную строку введите на странице https://www.coursera.org/learn/python-for-data-science/exam/fSPxW/sozdaniie-riekomiendatiel-noi-sistiemy Формат ответа - строка вида "X, X, X, X, X", где X - идентификаторы. Вводить ответ следует без кавычек.